In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [5]:
routes_df = pd.read_csv("/Users/mahinbindra/Downloads/Routes.csv")

In [6]:
# Initialize a dictionary to hold the costs
costs = {}

In [7]:
# Initialize a dictionary to hold sets of routes for each stop
routes_sets = {'A': set(), 'B': set(), 'C': set(), 'D': set(), 'E': set(), 'F': set()}

In [8]:
# Process each row in the dataframe
for index, row in routes_df.iterrows():
    # Extract the route ID (index in this case) and cost
    route_id = index + 1  # Adding 1 because route IDs are 1-indexed in the mathematical model
    costs[route_id] = row['Cost']
    
    # Extract the stops (removing brackets and spaces, then splitting by comma)
    stops = row['Routes'].strip("[]").replace(" ", "").split(',')
    
    # Map each stop in the route to its corresponding set in routes_sets
    for stop in stops:
        if stop.lower() == 'a':
            routes_sets['A'].add(route_id)
        elif stop.lower() == 'b':
            routes_sets['B'].add(route_id)
        elif stop.lower() == 'c':
            routes_sets['C'].add(route_id)
        elif stop.lower() == 'd':
            routes_sets['D'].add(route_id)
        elif stop.lower() == 'e':
            routes_sets['E'].add(route_id)
        elif stop.lower() == 'f':
            routes_sets['F'].add(route_id)

In [9]:
# Displaying the processed costs and routes_sets
costs, {key: list(val) for key, val in routes_sets.items()}  # Convert sets to lists for display purposes

({1: 136,
  2: 110,
  3: 90,
  4: 132,
  5: 118,
  6: 106,
  7: 136,
  8: 92,
  9: 132,
  10: 112,
  11: 80,
  12: 114,
  13: 134,
  14: 102,
  15: 68,
  16: 136,
  17: 118,
  18: 92,
  19: 72,
  20: 132,
  21: 114,
  22: 100,
  23: 88,
  24: 124,
  25: 104,
  26: 132,
  27: 120,
  28: 68,
  29: 136,
  30: 118,
  31: 74,
  32: 132,
  33: 114,
  34: 94,
  35: 138,
  36: 102,
  37: 80,
  38: 124,
  39: 110,
  40: 92,
  41: 120,
  42: 106,
  43: 88,
  44: 68,
  45: 114,
  46: 92,
  47: 74,
  48: 132,
  49: 118,
  50: 100,
  51: 138,
  52: 104,
  53: 90,
  54: 72,
  55: 134,
  56: 112,
  57: 94,
  58: 68},
 {'A': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   20,
   24,
   25,
   26,
   27,
   28,
   29,
   32,
   35,
   36,
   37,
   38,
   39,
   41,
   42,
   45,
   46,
   48,
   49,
   51,
   52,
   53,
   55,
   56],
  'B': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   9,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   2

In [10]:
# Create the model
m = Model("shuttle_service")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [11]:
# Create variables
# x[i] = 1 if route i is selected
x = m.addVars(range(1, 59), vtype=GRB.BINARY, name="x")
# y[i,j] = 1 if routes i and j both serve the Glendon campus
y = m.addVars([(i,j) for i in routes_sets['A'] for j in routes_sets['A'] if i > j], vtype=GRB.BINARY, name="y")
# z[k] = 1 if stop k is served by at least three routes
z = m.addVars(range(1, 7), vtype=GRB.BINARY, name="z")

In [12]:
# Set objective: Minimize the total cost of selected routes plus additional costs for Glendon campus, minus subsidies
m.setObjective(sum(costs[i] * x[i] for i in range(1, 59)) +
               sum(350 * y[i,j] for i,j in y.keys()) -
               sum(50 * z[k] for k in range(1, 7)), GRB.MINIMIZE)

In [13]:
# Add constraints to ensure at least one route serves each stop
for stop, set_of_routes in routes_sets.items():
    m.addConstr(sum(x[i] for i in set_of_routes) >= 1 + 2 * z[ord(stop) - ord('A') + 1], f"serve_{stop}")

# Add constraints for y variables
for i, j in y.keys():
    m.addConstr(y[i,j] <= x[i], f"y_constr_1_{i}_{j}")
    m.addConstr(y[i,j] <= x[j], f"y_constr_2_{i}_{j}")
    m.addConstr(y[i,j] >= x[i] + x[j] - 1, f"y_constr_3_{i}_{j}")

In [14]:
# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2346 rows, 844 columns and 5728 nonzeros
Model fingerprint: 0x265b4bfe
Variable types: 0 continuous, 844 integer (844 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 277260.00000
Found heuristic solution: objective 172.0000000
Presolve removed 1560 rows and 9 columns
Presolve time: 0.02s
Presolved: 786 rows, 835 columns, 2575 nonzeros
Variable types: 0 continuous, 835 integer (826 binary)

Root relaxation: objective 8.500000e+01, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [15]:
# Check if a feasible solution was found
if m.status == GRB.OPTIMAL:
    selected_routes = m.getAttr('X', x)
    selected_routes = [route for route, selected in selected_routes.items() if selected > 0.5]
    print("Selected Routes:", selected_routes)
    # Calculate and print the total cost of the selected routes
    total_cost = m.ObjVal
    print("Total Cost:", total_cost)
else:
    print("No optimal solution found")

Selected Routes: [18, 24, 40]
Total Cost: 108.0


In [16]:
route_with_largest_cost = max(costs, key=costs.get)
largest_cost = costs[route_with_largest_cost]
print(f"The route with the largest cost is {route_with_largest_cost} with a cost of ${largest_cost}")

The route with the largest cost is 35 with a cost of $138


In [18]:
# Cost per km
cost_per_km = 2.00

# Hypothetical distances between stops (you would replace these with the actual distances)
distance_U_a = 8 # distance from U to a
distance_a_f = 24 # distance from a to f
distance_f_U = 13 # distance from f to U

# Calculate the smallest cost for the route U -> a -> f -> U
smallest_cost_route = (distance_U_a + distance_a_f + distance_f_U) * cost_per_km

print(f"The smallest cost of the route U -> a -> f -> U is ${smallest_cost_route}")


The smallest cost of the route U -> a -> f -> U is $90.0


In [19]:
# Remove the subsidy from the objective function
m.setObjective(sum(costs[i] * x[i] for i in range(1, 59)) +
               sum(350 * y[i,j] for i,j in y.keys()), GRB.MINIMIZE)

# Reoptimize the model without the subsidy
m.optimize()

# After optimization, count the number of selected routes
selected_routes_count = sum(1 for i in x if x[i].X > 0.5)
print(f"Number of routes selected without subsidy: {selected_routes_count}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2346 rows, 844 columns and 5728 nonzeros
Model fingerprint: 0x8d275bb5
Variable types: 0 continuous, 844 integer (844 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 308

Presolve removed 2340 rows and 824 columns
Presolve time: 0.04s
Presolved: 6 rows, 20 columns, 82 nonzeros
Found heuristic solution: objective 124.0000000
Variable types: 0 continuous, 20 integer (20 binary)

Root relaxation: cutoff, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cut

In [21]:
total_x_variables = 58
total_y_variables = len(routes_sets['A']) * (len(routes_sets['A']) - 1) / 2
total_z_variables = 6

total_decision_variables = total_x_variables + total_y_variables + total_z_variables
print(total_decision_variables)


844.0
